In [ ]:
%matplotlib inline

Borealis Quickstart
=======================================

*Authors: Theodor Isacsson and Isaac De Vlugt*

In this quickstart, we will show how Xanadu's new Borealis quantum hardware can be programmed in minutes.
Borealis showcases quantum computational advantage, as can be seen in our publication in [Nature](https://xanadu.ai/qca-paper)[<sup>1</sup>](#advantage2022) --- meaning that it can compute in seconds quantities that would require years on the fastest available supercomputer.
It's hardware is based on [time-domain multiplexing (TDM)](https://strawberryfields.ai/photonics/demos/run_time_domain.html);
a single squeezed-light source emits batches of 216 time-ordered squeezed-light 
pulses that interfere with one another with the help of optical delay loops, 
programmable beamsplitters, and phase shifters. To demonstrate a quantum computational
advantage, we chose a task that is proven to be computationally hard:
[Gaussian Boson Sampling (GBS)](https://pennylane.ai/qml/demos/tutorial_gbs.html).
We will show you how to run our experiment!


To start off, we import the necessary packages. Then, by loading
a `device <strawberryfields.Device>` object which contains relevant
and up-to-date information about Borealis, we can retrieve the device
by calling the `device <strawberryfields.Device>`
property on the `RemoteEngine <strawberryfields.RemoteEngine>` object.



In [ ]:
import strawberryfields as sf
import numpy as np

eng = sf.RemoteEngine("borealis")
device = eng.device

A time-domain program is defined by arguments that each quantum
gate applies at each time bin, within the duration of the program.
We're going to use helper functions that will do some heavy lifting for us.
The `borealis_gbs <strawberryfields.tdm.borealis_gbs>` function
will summarize all the necessary information for Strawberry Fields
and our hardware to implement your circuit, then
`get_mode_indices <strawberryfields.tdm.get_mode_indices>`
gives us the number of concurrent modes ``N`` and the mode indices ``n``.




In [ ]:
from strawberryfields.tdm import borealis_gbs, get_mode_indices

gate_args_list = borealis_gbs(device, modes=216, squeezing="high")
delays = [1, 6, 36]
n, N = get_mode_indices(delays)

Finally, we can construct and run the circuit.




In [ ]:
from strawberryfields.ops import Sgate, Rgate, BSgate, MeasureFock

prog = sf.TDMProgram(N)

with prog.context(*gate_args_list) as (p, q):
    Sgate(p[0]) | q[n[0]]
    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]])
    MeasureFock() | q[0]

shots = 10_000
results = eng.run(prog, shots=shots, crop=True)
print(results.samples)

Congratulations for successfully demonstrating quantum advantage!




<h2>References</h2>


<span id="advantage2022">[1]</span>: <i>Madsen, L.S., Laudenbach, F., Askarani, M.F. et al.</i> "Quantum computational advantage with a programmable photonic processor", <a href="https://www.nature.com/articles/s41586-022-04725-x">Nature 606, 75-81</a>, 2022

About the authors
-----------------
<div class="bio" style="display: flex">
    <div class="photo" style="max-width: 60px; margin-right: 1em;">
        <img class="photo__img" src="./.imgs/Theodor.jpg" alt="Theodor Isacsson" style="width: 100%; border-radius: 50%;">
    </div>
    <div class="bio-text">
        <h4 class="bio-text__author-name" style="margin-top: 0; margin-bottom: 0; font-weight: bold;">Theodor Isacsson</h4>
        <p class="bio-text__author-description" style="margin-top: 1em;">Theodor is a quantum software developer at Xanadu. He works on the
            photonics software stack, leading the efforts to expand and maintain packages such as Strawberry Fields and
            The Walrus.</p>
    </div>
</div>
<div class="bio" style="display: flex">
    <div class="photo" style="max-width: 60px; margin-right: 1em;">
        <img class="photo__img" src="./.imgs/Isaac.png" alt="Isaac De Vlugt" style="width: 100%; border-radius: 50%;">
    </div>
    <div class="bio-text">
        <h4 class="bio-text__author-name" style="margin-top: 0; margin-bottom: 0; font-weight: bold;">Isaac De Vlugt</h4>
        <p class="bio-text__author-description" style="margin-top: 1em;">Isaac De Vlugt is a quantum computing educator at Xanadu. His work involves creating accessible quantum computing content for the community, as well as spamming gifs in our Slack channels.</p>
    </div>
</div>